In [1]:
# Import library yang dibutuhkan
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import joblib

# Untuk preprocessing dan feature engineering
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Set random seed untuk reproducibility
np.random.seed(42)

# Load dataset
df = pd.read_csv('insurance.csv')

# ===============================================================
# 1. EXPLORATORY DATA ANALYSIS (EDA)
# ===============================================================
print("Dataset Overview:")
print(f"Shape: {df.shape}")
print(df.info())
print("\nDescriptive Statistics:")
print(df.describe())

# Cek missing values
print("\nMissing Values:")
print(df.isnull().sum())

# Visualisasi distribusi target (charges)
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
sns.histplot(df['charges'], kde=True)
plt.title('Distribusi Charges')
plt.xlabel('Charges')

plt.subplot(1, 2, 2)
sns.histplot(np.log1p(df['charges']), kde=True)
plt.title('Distribusi Log(Charges)')
plt.xlabel('Log(Charges)')

plt.tight_layout()
plt.savefig('charges_distribution.png')
plt.close()

# Cek normalitas distribusi target (charges)
_, p_value = stats.shapiro(df['charges'].sample(min(1000, len(df))))  # Sample untuk menghindari error dengan dataset besar
print(f"\nShapiro-Wilk Test p-value for charges: {p_value}")
print(f"Is charges normally distributed? {'Yes' if p_value > 0.05 else 'No'}")

# Visualisasi korelasi antar variabel numerik
numeric_cols = ['age', 'bmi', 'children', 'charges']
plt.figure(figsize=(10, 8))
correlation_matrix = df[numeric_cols].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Matrix')
plt.savefig('correlation_matrix.png')
plt.close()

# Visualisasi pengaruh fitur kategorikal terhadap charges
plt.figure(figsize=(15, 10))

plt.subplot(2, 2, 1)
sns.boxplot(x='sex', y='charges', data=df)
plt.title('Charges by Sex')

plt.subplot(2, 2, 2)
sns.boxplot(x='smoker', y='charges', data=df)
plt.title('Charges by Smoker Status')

plt.subplot(2, 2, 3)
sns.boxplot(x='region', y='charges', data=df)
plt.title('Charges by Region')

plt.subplot(2, 2, 4)
sns.boxplot(x='children', y='charges', data=df)
plt.title('Charges by Number of Children')

plt.tight_layout()
plt.savefig('categorical_features_impact.png')
plt.close()

# Visualisasi scatter plot antara fitur numerik dan charges
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
sns.scatterplot(x='age', y='charges', hue='smoker', data=df)
plt.title('Age vs Charges')

plt.subplot(1, 3, 2)
sns.scatterplot(x='bmi', y='charges', hue='smoker', data=df)
plt.title('BMI vs Charges')

plt.subplot(1, 3, 3)
sns.scatterplot(x='children', y='charges', hue='smoker', data=df)
plt.title('Children vs Charges')

plt.tight_layout()
plt.savefig('numerical_features_impact.png')
plt.close()

# ===============================================================
# 2. FEATURE ENGINEERING (MANUAL, NO CUSTOM TRANSFORMERS)
# ===============================================================
# We'll use a more direct approach without custom transformers

# Create a DataFrame for engineered features
df_engineered = df.copy()

# 1. Add Health Risk Score
# Age risk: 1 point for every 10 years after 18
df_engineered['age_risk'] = (df_engineered['age'] - 18) / 10

# BMI risk: points based on BMI category
df_engineered['bmi_risk'] = 0
# Underweight (BMI < 18.5)
df_engineered.loc[df_engineered['bmi'] < 18.5, 'bmi_risk'] = 1
# Normal (18.5 <= BMI < 25)
df_engineered.loc[(df_engineered['bmi'] >= 18.5) & (df_engineered['bmi'] < 25), 'bmi_risk'] = 0
# Overweight (25 <= BMI < 30)
df_engineered.loc[(df_engineered['bmi'] >= 25) & (df_engineered['bmi'] < 30), 'bmi_risk'] = 1
# Obese (BMI >= 30)
df_engineered.loc[df_engineered['bmi'] >= 30, 'bmi_risk'] = 2

# Smoker risk: 3 points if smoker
df_engineered['smoker_risk'] = np.where(df_engineered['smoker'] == 'yes', 3, 0)

# Total health risk score
df_engineered['health_risk_score'] = df_engineered['age_risk'] + df_engineered['bmi_risk'] + df_engineered['smoker_risk']

# 2. Feature Interactions
df_engineered['smoker_binary'] = np.where(df_engineered['smoker'] == 'yes', 1, 0)
df_engineered['age_smoker'] = df_engineered['age'] * df_engineered['smoker_binary']
df_engineered['bmi_smoker'] = df_engineered['bmi'] * df_engineered['smoker_binary']
df_engineered['bmi_age'] = df_engineered['bmi'] * df_engineered['age'] / 100

# 3. BMI Categories
df_engineered['bmi_category'] = 'normal'
df_engineered.loc[df_engineered['bmi'] < 18.5, 'bmi_category'] = 'underweight'
df_engineered.loc[(df_engineered['bmi'] >= 25) & (df_engineered['bmi'] < 30), 'bmi_category'] = 'overweight'
df_engineered.loc[df_engineered['bmi'] >= 30, 'bmi_category'] = 'obese'

# 4. Age Groups
df_engineered['age_group'] = 'middle'
df_engineered.loc[df_engineered['age'] < 35, 'age_group'] = 'young'
df_engineered.loc[df_engineered['age'] >= 50, 'age_group'] = 'senior'

# 5. Target Encoding (we'll do this later with train/test split)

# ===============================================================
# 3. PREPROCESSING AND PREPARING DATA
# ===============================================================
# Define features and target
features = ['age', 'sex', 'bmi', 'children', 'smoker', 'region', 
            'health_risk_score', 'age_smoker', 'bmi_smoker', 'bmi_age', 
            'bmi_category', 'age_group']
target = 'charges'

X = df_engineered[features]
y = df_engineered[target]

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Check distribution of charges to determine appropriate scaler
_, p_value_charges = stats.shapiro(y_train.sample(min(1000, len(y_train))))
print(f"\nShapiro-Wilk Test p-value for charges (training set): {p_value_charges}")
use_standard_scaler = (p_value_charges > 0.05)
print(f"Using StandardScaler: {use_standard_scaler}")

# 5. Apply Target Encoding on categorical columns
categorical_cols = ['sex', 'smoker', 'region', 'bmi_category', 'age_group']
for col in categorical_cols:
    # Calculate mean target value for each category in training data
    encoding_map = y_train.groupby(X_train[col]).mean().to_dict()
    
    # Apply encoding to both train and test data
    X_train[f'{col}_encoded'] = X_train[col].map(encoding_map)
    X_test[f'{col}_encoded'] = X_test[col].map(encoding_map)
    
    # Handle any missing values in test set (categories not seen in training)
    if X_test[f'{col}_encoded'].isna().any():
        global_mean = y_train.mean()
        X_test[f'{col}_encoded'].fillna(global_mean, inplace=True)

# Define preprocessing pipeline
numeric_features = ['age', 'bmi', 'children', 'health_risk_score', 
                    'age_smoker', 'bmi_smoker', 'bmi_age']
categorical_features = ['sex', 'smoker', 'region', 'bmi_category', 'age_group']
encoded_features = [f'{col}_encoded' for col in categorical_features]

# Choose scaler based on distribution
scaler = StandardScaler() if use_standard_scaler else MinMaxScaler()

# Preprocessing pipeline
numeric_transformer = Pipeline(steps=[
    ('scaler', scaler)
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(drop='first', sparse_output=False))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
        ('encoded', 'passthrough', encoded_features)
    ])

# Apply preprocessing
preprocess_pipeline = Pipeline(steps=[('preprocessor', preprocessor)])
preprocess_pipeline.fit(X_train)

X_train_preprocessed = preprocess_pipeline.transform(X_train)
X_test_preprocessed = preprocess_pipeline.transform(X_test)

# Apply log transformation to target (charges) if not normally distributed
if not use_standard_scaler:
    print("Applying log transformation to charges...")
    y_train_log = np.log1p(y_train)
    y_test_log = np.log1p(y_test)
else:
    y_train_log = y_train
    y_test_log = y_test

# ===============================================================
# 4. MODEL TRAINING AND EVALUATION
# ===============================================================
# Create and train model
model = LinearRegression()
model.fit(X_train_preprocessed, y_train_log)

# Make predictions
y_train_pred_log = model.predict(X_train_preprocessed)
y_test_pred_log = model.predict(X_test_preprocessed)

# If log transformation was applied, reverse it for evaluation
if not use_standard_scaler:
    y_train_pred = np.expm1(y_train_pred_log)
    y_test_pred = np.expm1(y_test_pred_log)
else:
    y_train_pred = y_train_pred_log
    y_test_pred = y_test_pred_log

# Evaluate model
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

print("\nModel Evaluation:")
print(f"Training RMSE: {train_rmse:.2f}")
print(f"Test RMSE: {test_rmse:.2f}")
print(f"Training R²: {train_r2:.4f}")
print(f"Test R²: {test_r2:.4f}")

# Cross-validation for model stability
# We'll use a simple approach with the preprocessed data
cv_scores = cross_val_score(model, X_train_preprocessed, y_train_log, 
                           cv=5, scoring='neg_root_mean_squared_error')
cv_rmse = -cv_scores.mean()
print(f"\nCross-Validation RMSE: {cv_rmse:.2f}")

# Plot actual vs predicted values
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_test_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel('Actual Charges')
plt.ylabel('Predicted Charges')
plt.title('Actual vs Predicted Charges')
plt.savefig('actual_vs_predicted.png')
plt.close()

# Plot residuals
residuals = y_test_pred - y_test
plt.figure(figsize=(10, 6))
plt.scatter(y_test_pred, residuals, alpha=0.5)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicted Charges')
plt.ylabel('Residuals')
plt.title('Residual Plot')
plt.savefig('residuals.png')
plt.close()

# Feature importance analysis
# Get feature names from preprocessing pipeline
num_features = numeric_features
cat_features = []
for name, _, cols in preprocessor.transformers_:
    if name == 'cat':
        cat_encoder = preprocessor.named_transformers_['cat'].named_steps['onehot']
        cat_features = list(cat_encoder.get_feature_names_out(categorical_features))
encoded_features = [f for f in encoded_features]

all_features = num_features + cat_features + encoded_features

# Create a DataFrame of coefficients
coef_df = pd.DataFrame({'Feature': all_features[:len(model.coef_)], 'Coefficient': model.coef_})
coef_df = coef_df.sort_values('Coefficient', ascending=False)

plt.figure(figsize=(12, 8))
sns.barplot(x='Coefficient', y='Feature', data=coef_df.head(15))
plt.title('Top 15 Important Features')
plt.tight_layout()
plt.savefig('important_features.png')
plt.close()

# ===============================================================
# 5. SAVE MODEL AND PREPROCESSING PIPELINE
# ===============================================================
# Create a class to handle prediction
class InsurancePredictionModel:
    def __init__(self, model, preprocessor, categorical_cols, encoding_maps, log_transform=False):
        self.model = model
        self.preprocessor = preprocessor
        self.categorical_cols = categorical_cols
        self.encoding_maps = encoding_maps
        self.log_transform = log_transform
        self.global_mean = None  # Will be set when saving
    
    def prepare_features(self, data):
        """
        Prepare features including engineered features
        """
        if isinstance(data, dict):
            data = pd.DataFrame([data])
            
        df = data.copy()
        
        # 1. Health Risk Score
        df['age_risk'] = (df['age'] - 18) / 10
        
        # BMI risk
        df['bmi_risk'] = 0
        df.loc[df['bmi'] < 18.5, 'bmi_risk'] = 1
        df.loc[(df['bmi'] >= 18.5) & (df['bmi'] < 25), 'bmi_risk'] = 0
        df.loc[(df['bmi'] >= 25) & (df['bmi'] < 30), 'bmi_risk'] = 1
        df.loc[df['bmi'] >= 30, 'bmi_risk'] = 2
        
        # Smoker risk
        df['smoker_risk'] = np.where(df['smoker'] == 'yes', 3, 0)
        
        # Total health risk score
        df['health_risk_score'] = df['age_risk'] + df['bmi_risk'] + df['smoker_risk']
        
        # 2. Feature Interactions
        df['smoker_binary'] = np.where(df['smoker'] == 'yes', 1, 0)
        df['age_smoker'] = df['age'] * df['smoker_binary']
        df['bmi_smoker'] = df['bmi'] * df['smoker_binary']
        df['bmi_age'] = df['bmi'] * df['age'] / 100
        
        # 3. BMI Categories
        df['bmi_category'] = 'normal'
        df.loc[df['bmi'] < 18.5, 'bmi_category'] = 'underweight'
        df.loc[(df['bmi'] >= 25) & (df['bmi'] < 30), 'bmi_category'] = 'overweight'
        df.loc[df['bmi'] >= 30, 'bmi_category'] = 'obese'
        
        # 4. Age Groups
        df['age_group'] = 'middle'
        df.loc[df['age'] < 35, 'age_group'] = 'young'
        df.loc[df['age'] >= 50, 'age_group'] = 'senior'
        
        # 5. Apply target encoding
        for col in self.categorical_cols:
            encoding_map = self.encoding_maps.get(col, {})
            df[f'{col}_encoded'] = df[col].map(encoding_map)
            
            # Handle unseen categories
            if df[f'{col}_encoded'].isna().any():
                df[f'{col}_encoded'].fillna(self.global_mean, inplace=True)
                
        return df
    
    def predict(self, data):
        """
        Make predictions for new data
        """
        # Prepare features
        df_prepared = self.prepare_features(data)
        
        # Transform using preprocessing pipeline
        X_transformed = self.preprocessor.transform(df_prepared)
        
        # Make prediction
        pred_log = self.model.predict(X_transformed)
        
        # Transform back if needed
        if self.log_transform:
            pred = np.expm1(pred_log)
        else:
            pred = pred_log
            
        return pred

# Create encoding maps
encoding_maps = {}
categorical_cols = ['sex', 'smoker', 'region', 'bmi_category', 'age_group']
for col in categorical_cols:
    encoding_maps[col] = y_train.groupby(X_train[col]).mean().to_dict()

# Create prediction model
insurance_model = InsurancePredictionModel(
    model=model,
    preprocessor=preprocess_pipeline,
    categorical_cols=categorical_cols,
    encoding_maps=encoding_maps,
    log_transform=(not use_standard_scaler)
)

# Set global mean for unseen categories
insurance_model.global_mean = y_train.mean()

# Save model and pipeline
joblib.dump(insurance_model, 'insurance_prediction_model.joblib')

print("\nModel saved successfully!")

# ===============================================================
# 6. DEPLOYMENT EXAMPLE
# ===============================================================
print("\nExample of model deployment code:")
print("""
# Code for model deployment
import joblib
import pandas as pd

def predict_insurance_charges(data):
    '''
    Predict insurance charges for new data
    
    Parameters:
    -----------
    data : dict or pd.DataFrame
        Data containing features: age, sex, bmi, children, smoker, region
    
    Returns:
    --------
    float : Predicted insurance charges
    '''
    # Load the model
    model = joblib.load('insurance_prediction_model.joblib')
    
    # Make prediction
    prediction = model.predict(data)
    
    return prediction[0] if len(prediction) == 1 else prediction

# Example usage
new_customer = {
    'age': 35,
    'sex': 'male',
    'bmi': 27.5,
    'children': 2,
    'smoker': 'no',
    'region': 'northeast'
}

predicted_charge = predict_insurance_charges(new_customer)
print(f"Predicted insurance charge: ${predicted_charge:.2f}")
""")

# Test the saved model
print("\nTesting saved model with a sample customer:")
new_customer = {
    'age': 35,
    'sex': 'male',
    'bmi': 27.5,
    'children': 2,
    'smoker': 'no',
    'region': 'northeast'
}

loaded_model = joblib.load('insurance_prediction_model.joblib')
prediction = loaded_model.predict(new_customer)
print(f"Predicted insurance charge: ${prediction[0]:.2f}")

# ===============================================================
# 7. ANALISIS HASIL
# ===============================================================
print("\n===== ANALISIS HASIL =====")

print("\n1. Analisis Distribusi Data")
if not use_standard_scaler:
    print("- Variabel target 'charges' tidak berdistribusi normal (p-value < 0.05)")
    print("- Menggunakan transformasi logaritma pada variabel target")
    print("- Menggunakan MinMaxScaler untuk feature scaling")
else:
    print("- Variabel target 'charges' berdistribusi normal (p-value > 0.05)")
    print("- Tidak perlu transformasi logaritma")
    print("- Menggunakan StandardScaler untuk feature scaling")

print("\n2. Analisis Performa Model")
print(f"- RMSE pada data test: {test_rmse:.2f}")
print(f"- R² pada data test: {test_r2:.4f}")
print(f"- Nilai R² {test_r2:.4f} menunjukkan bahwa model dapat menjelaskan {test_r2*100:.2f}% variasi pada biaya asuransi")

if test_r2 >= 0.75:
    print("- Model memiliki kemampuan prediksi yang baik (R² > 0.75)")
elif test_r2 >= 0.5:
    print("- Model memiliki kemampuan prediksi yang cukup (R² > 0.5)")
else:
    print("- Model memiliki kemampuan prediksi yang kurang baik (R² < 0.5)")

print("\n3. Analisis Faktor-faktor yang Mempengaruhi Biaya Asuransi")
print("- Berdasarkan koefisien model, faktor yang paling berpengaruh adalah:")
for i in range(min(5, len(coef_df))):
    print(f"  {i+1}. {coef_df.iloc[i]['Feature']}: {coef_df.iloc[i]['Coefficient']:.4f}")

print("\n4. Insight untuk Bisnis")
print("- Status merokok merupakan faktor yang sangat signifikan dalam menentukan biaya asuransi")
print("- BMI (indeks massa tubuh) memiliki pengaruh positif terhadap biaya asuransi")
print("- Usia juga berpengaruh positif, semakin tua seseorang semakin tinggi biaya asuransinya")
print("- Interaksi antara usia dan status merokok sangat berpengaruh pada biaya asuransi")

print("\n5. Rekomendasi")
print("- Model ini dapat digunakan untuk estimasi biaya asuransi calon nasabah")
print("- Feature engineering, terutama interaksi antara faktor risiko, sangat membantu meningkatkan akurasi prediksi")
print("- Untuk deployment, gunakan model yang telah disimpan untuk memastikan preprocessing konsisten")
print("- Pertimbangkan untuk melakukan update model secara berkala dengan data terbaru untuk mempertahankan akurasi")   

Dataset Overview:
Shape: (1338, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB
None

Descriptive Statistics:
               age          bmi     children       charges
count  1338.000000  1338.000000  1338.000000   1338.000000
mean     39.207025    30.663397     1.094918  13270.422265
std      14.049960     6.098187     1.205493  12110.011237
min      18.000000    15.960000     0.000000   1121.873900
25%      27.000000    26.296250     0.000000   4740.287150
50%      39.000000    30.400000     1.000000   9382.033000
75